In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#pandas библиотека для работы с табличками, мы будем ее использовать чтобы загрузить данные и все
import pandas as pd

Загружаем данные

In [ ]:
data = pd.read_csv("./data/train.csv")

In [ ]:
data.head()

Вытаскиваем из таблички x и y

In [ ]:
x_train = data[data.columns[1:]].values
y_train = data[data.columns[0]].values

In [ ]:
print('shapes: ', x_train.shape, y_train.shape)

В данных 42000 примеров, у каждого 784 признака. Эти признаки (pixel0 - pixel783) это просто пиксели картинки 28х28, с изображением цифры, растянутые в вектор длиной 28*28

Рассмотрим пример цифры из данных. 

In [ ]:
digit_example = x_train[239]
label_example = y_train[239]

Зарешейпите digit_example, так чтобы отобразить его как картинку

In [ ]:
digit_example_square = *your code here*

In [ ]:
plt.imshow(digit_example_square)
plt.show()

Постройте по одному примеру каждой цифры. 
Для этого надо построить маску тех мест, где в Y определенная цифра, и построить любой из соответствующих X.

In [ ]:
for i in range(10):
    
    *your code here*
    
    digit_example = *your code here*
    
    print('Shape: ', digit_example.shape)
    
    plt.imshow(digit_example)
    plt.show()

### Классификация из двух классов

Построим классификатор, отличающий две цифры. 

Выделите из датасета кусок соответсвующий любым двум цифрам.

In [ ]:
x_train_subsample = *your code here*
y_train_subsample = *your code here*

Импортируем модель логистической регрессии из sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression

В LogisticRegression по умолчанию вшита регуляризация.

Параметр отвечающий за регуляризацию называется penalty и бывает:
- 'l1' для регуляризации с помощью суммы модулей (как lasso)
- 'l2' для регуляризации с помощью суммы квадратов (как ridge)

Величина регуляризации характеризуется параметром C. Будьте внимательны. В докстринге про С написано:  
С : float, default: 1.0
     Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

Это говорит о том, что С - обратный коэф регуляризации. Больше С, меньше регуляризация.

In [ ]:
lr = LogisticRegression(penalty = 'l2', C = 1)

Проверьте логрегрессию на двух классах с помощью разбиения на train\test или кросс валидации(может быть медленно)

In [ ]:
from sklearn.model_selection import train_test_split, KFold

In [ ]:
predictions = []

*your code here*

Посчитайте roc_auc_score. 

Можно так же построить график roc кривой с помощью roc_curve и plt.plot()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
*your code here*

In [ ]:
print('Model CV AUC score', *your code here*)

Попробуйте настроить параметр принятия решения с помощью кросс валидации или train\test разбиения. Для этого воспользуйтесь методом lr.predict_proba(). Данный метод вернет вероятности обоих классов для каждого примера, и можно будет все что больше определенного порога отнести к одному классу, все что меньше к другому. 

Ниже реализация без кросс валидации. Ваша задача для разных порогов обернуть этот кусок кода в кросс валидацию и выбрать оптимальный порог для данной пары. 

In [ ]:
proba = lr.predict_proba()

treashold = 0.5
predictions = (proba[:, 1] > 0.5).astype(int)



### Классификация OneVsAll


Данных много, так что обучить модель может быть не быстро. Чтобы ускорить процесс, можно для начала проредить выборку в 2-5 раз, взяв каждый  k-ый пример

In [ ]:
x_train_rare = *your code here*
y_train_rare = *your code here*

Если после этого модель все еще будет учиться долго (больше 5-7 мин). Можно проредить еще и сами пиксели в 2 или 3 раза. 

Для этого надо сначала зарешейпить 784 в 28х28, потом вязть каждый n-ый пиксель, чтобы получить 9х9 и наконец зарешейпить назад в 81

In [ ]:
x_train_rare_small = *your code here*

Подготовим train_test_split.

In [ ]:
x_train_mlcs, y_train_mlcs =  *your code here*
x_test_mlcs, y_test_mlcs =  *your code here*

Проверьте, что в трейне и в тесте приблизительно поровну цифр всех видов. 

In [ ]:
for i in range(10):
    print('train labels ', str(i), str((y_train_mlcs == i).sum()))
    print('test labels ', str(i), str((y_test_mlcs == i).sum()))
    

Если цифры не равномерно распределились, то нужно повторить разбиение предварително перемешав массивы x_train и y_train.

Для этого следует взять индексы, перемешать их, а потом вязть x и y в таком порядке

In [ ]:
idxs = np.arange(x_train_rare_small.shape[0])
np.random.shuffle(idxs)

x_train_rare_small_shuffle =  *your code here*
y_train_rare_shuffle =  *your code here*

x_train_mlcs, y_train_mlcs =  *your code here*
x_test_mlcs, y_test_mlcs =  *your code here*

Теперь данные готовы

В sklearn есть готовая обертка для многоклассовой классификации

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

Для использования OneVsRestClassifier надо сначала создать базовый классификатор, и передать его первым аргументом в OneVsRest. У OneVsRest есть парамерт n_jobs, который регулирует кол-во параллельных процессов обучения. Имеет смысл поставить количество ядер на машине, или кол-во ядер -1

In [ ]:
lr =  *your code here*
ovr_clsf = OneVsRestClassifier(lr, n_jobs=8)

Теперь учим модель и предсказываем

In [ ]:
ovr_clsf.fit()
pred = ovr_clsf.predict()


Если цифры распределенны равномерно, то accuracy - неплохая метрика. Другой неплохой способ оценки - матрица ошибок, confusion matrix. Это матрица, в которой $M_{ij}$ это кол-во примеров с лэйблом i, предсказанных как j.

Посчитайте обе этих метрики

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

Опишите какие цифры модель хуже всего отличает друг от друга.

Еще можно нарисовать несколько примеров цифр, в которых модель ошиблась. Вероятно у вас тоже возникнут проблемы с ними.

Чтобы получить хорошую оценку модели есть два варианта.

1. Для каждого класса посчитать precision, recall и FPR и усреднить. Это называется макрометрики.
2. Считать не средние precision, recall и FPR, а сделать их сразу усредненными, это называется микрометрики. 
Например вот такой будет precision

     $$ Precision_{micro} = \frac{TP_1 + ... TP_k}{TP_1 + ... TP_k + FP_1 + ... FP_k}$$

https://www.quora.com/What-are-some-good-error-metrics-for-multi-class-classification-when-you-have-many-objects-to-classify

Если есть желание, можно посчитать такие метрики. 

### Предсказания. 

С помощью лучшей модели можно построить предсказания. Проверить что там предсказалось можно на сайте https://www.kaggle.com/c/digit-recognizer/leaderboard
Там есть инструкция в каком формате нужно представить данные для отправки решения. Чтобы сохранять файл в формате csv (coma separated value), есть специальные функции в pandas

Это соревнование по классификации цифр и наша модель там не должна быть супер хороша пока, тем не меннее попробовать отправить вполне можно. 

Ниже набросок правильной обработки данных для отправки. 

In [ ]:
test_data = pd.read_csv("./data/test.csv")

In [ ]:
test_x = test_data.values

In [ ]:
#make predictions
*your code here*

In [ ]:
idxs = test_data.index.values

In [ ]:
pred_w_indexes = np.hstack((idxs.reshape(-1, 1), pred.reshape(-1, 1)))

pred_as_pandas_data_frame = pd.DataFrame(pred_w_indexes, columns = ['ImageId','Label'])

pred_as_pandas_data_frame .to_csv('pred.csv')